# Asg-5


* Build your own calculator
  - The code is adapted slightly from https://www.dabeaz.com/ply/example.html

  - further seal knowledge of lexers (RE based) and parsers (CFG based) in your minds 

In [ ]:

#----TURN THESE ON IF RUNNING ON JOVE ON COLAB BY REMOVING THE ''' ... '''

'''
! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
import sys
sys.path.append('./Jove')
sys.path.append('./Jove/jove')

 
from lex import *
from yacc import *
import math 
'''

#----TURN THESE ON IF RUNNING ON OWN INSTALLATION (default turned-on)

import sys
sys.path[0:0] = ['../..',       '../../3rdparty',
                 '../../..',    '../../../3rdparty',
                 '../../../..', '../../../../3rdparty'
                ] # Put these at the head of the search path

from lex import *
from yacc import *
import math

In [ ]:
tokens = (
    'NAME','NUMBER',
    'PLUS','MINUS','TIMES','DIVIDE','EQUALS',
    'LPAREN','RPAREN', # <== ADD 'FAC here'
    )

# Tokens

t_PLUS    = r'\+'
t_MINUS   = r'-'
t_TIMES   = r'\*'
t_DIVIDE  = r'/'
t_EQUALS  = r'='
t_LPAREN  = r'\('
t_RPAREN  = r'\)'
t_NAME    = r'[a-zA-Z_][a-zA-Z0-9_]*'
# t_FAC   = # <== ADD definition for t_FAC here
 

def t_NUMBER(t):
    r'\d+'   # <== Extend the regexp to handle fractions
    try:
        if '.' in t.value:
            t.value = float(t.value)
        else:
            t.value = int(t.value)
    except ValueError:
        print("Number value too large %d", t.value)
        t.value = 0
    return t

# Ignored characters
t_ignore = " \t"

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count("\n")
    
def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)
    

# Parsing rules


# dictionary of names
names = { }

def p_statement_assign(t):
    'statement : NAME EQUALS expression'
    # The full calculator logic is driven by processing statement at a time
    # and either storing expressions in new variables in the "names" dictionary...
    names[t[1]] = t[3]

def p_statement_expr(t):
    'statement : expression'
    # ... or printing the value of the expression represented by it
    print(t[1])
                
def p_expression(t):
    '''expression : expression PLUS term
                  | expression MINUS term'''
    if t[2] == '+' :  t[0] = t[1] + t[3]
    elif t[2] == '-': t[0] = t[1] - t[3]
    else: print("expression goes to unknown item.")

def p_expression_T(t):
    '''expression : term'''
    t[0] = t[1]

'''
def p_expression_name(t):
    'expression : NAME'
    try:
        t[0] = names[t[1]]
    except LookupError:
        print("Undefined name '%s'" % t[1])
        t[0] = 0
'''

def p_term(t):
    '''term : term TIMES factor
            | term DIVIDE factor'''
    if t[2] == '*' :  t[0] = t[1] * t[3]
    elif t[2] == '/': t[0] = t[1] / t[3]
    else: print("term goes to unknown item.")
    
def p_term_F(t):
    '''term : factor'''
    t[0] = t[1]
    
def p_factor(t):
    '''factor : MINUS factor
              | LPAREN expression RPAREN
              | NUMBER
              | NAME
              ''' # <== ADD FAC factor as another production here
              
    if t[1] == '-' :  t[0] = -t[2]  
    elif t[1] == '(': t[0] = t[2]
    elif t[1] == '!': 
        if (type(t[2]) == int):
            if (t[2] >= 0):
                t[0] = math.factorial(t[2])
            else:
                print("Factorial of a negative number is undefined!")
                t.lexer.skip(1)
                
        else:
            print("Factorial of a fraction is undefined!")
            t.lexer.skip(1)
            
    else:
        if type(t[1]) == int or type(t[1]) == float:
            t[0] = t[1]
        else:
            try:
                t[0] = names[t[1]]
            except LookupError:
                print("Undefined name '%s'" % t[1])
                t[0] = 0
 
def p_error(t):
    print("Syntax error at '%s'" % t.value)
    
calclexer = lex()
calcparser = yacc()

while True:
    try:
        s = input('calc > ')   # Use raw_input on Python 2
    except EOFError:
        break
    if (s=='END'):
        break
    calcparser.parse(s, lexer=calclexer)

Your output must resemble the above.